In [3]:
import pandas as pd 
import numpy as np

# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


# Method 1: Using Pandas (Recommended for data cleaning)
# Read the Excel file
df = pd.read_excel("E:\\Lenguard-Second project\\rent_roll_files\\Abbie Lakes - Rent Roll - 5.15.24 (Lender Pulled) ajr.xlsx")

# Remove rows where all values are NaN
df_clean = df.dropna(how='all')

# Remove columns where all values are NaN
df_clean = df_clean.dropna(axis=1, how='all')

df_clean

property_info = df_clean.iloc[0:4]

# Get the first column with property info headers
property_info_headers = property_info['Abbie Lakes'].values
print("Property Information :")
for header in property_info_headers:
    print(header)


rent_roll_data = df_clean.iloc[4:]

# Set the 4th row (index 4) as the header for rent roll data
rent_roll_data.columns = rent_roll_data.iloc[0]

# Remove the header row from the data
rent_roll_data = rent_roll_data.iloc[1:].reset_index(drop=True)

# Find rows containing 'total' (case insensitive)
total_rows = rent_roll_data[rent_roll_data.apply(lambda x: x.astype(str).str.contains('total', case=False)).any(axis=1)]

# # Remove total rows from main data
rent_roll_data = rent_roll_data[~rent_roll_data.index.isin(total_rows.index)]

total_rows.iloc[:1]

# Get the first row of total_rows and drop NaN values
total_row_clean = total_rows.iloc[0].dropna()

# Create a new DataFrame with just the non-NaN values
total_summary = pd.DataFrame([total_row_clean]).reset_index(drop=True)

# Display the cleaned total row
print("\nCleaned Total Row Summary:")
print(total_summary)



Property Information :
RENT ROLL DETAIL MODIFIED
5/16/2024 3:11:55 AM
As of Date: 5/15/2024
Parameters: Property - ALL; Unit Designation - ALL; Subjournal - ALL; Sort By - Unit; Display - Market + Addl.

Cleaned Total Row Summary:
4 Resh ID Market + Addl. Dep On Hand    Balance Total Charges  \
0   TOTAL     285,137.00   60,189.83  -1,774.23    261,921.20   

4 RENT                           PEST                            \
0                     251,448.00                         630.00   

4 VALET TRASH                    TRASH                           \
0                       6,374.00                       1,682.00   

4 PETRENT                        LOP15                           \
0                       1,825.00                         795.00   

4 EMPLCRED                       OHIOCHOICE                      \
0                        -657.80                           0.00   

4 WAIVED ADMIN FEE               WAIVED APP FEE                  \
0                        -150.0

In [ ]:
rent_roll_data

# Rent Roll Processor
#
# Functions:
# - load_excel: Load file
# - parse_data: Convert to df
# - find_headers: Get headers
# - get_metadata: Property info
# - process_data: Clean data
#
# Purpose: Extract property & rent roll data


In [65]:
! pip install "pydantic>=2.0.0" "openai>=1.0.0"

  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 27.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.3
    Uninstalling pydantic_core-2.23.3:
      Successfully uninstalled pydantic_core-2.23.3
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.12
    Uninstalling pydantic-1.10.12:
      Successfully uninstalled pydantic-1.10.12


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.10.6 which is incompatible.
langchain-core 0.3.0 requires jsonpatch<2.0,>=1.33, but you have jsonpatch 1.32 which is incompatible.
langchain-core 0.3.0 requires packaging<25,>=23.2, but you have packaging 23.1 which is incompatible.
scrapegraphai 1.20.1 requires beautifulsoup4>=4.12.3, but you have beautifulsoup4 4.12.2 which is incompatible.
scrapegraphai 1.20.1 requires python-dotenv>=1.0.1, but you have python-dotenv 0.21.0 which is incompatible.
scrapegraphai 1.20.1 requires tqdm>=4.66.4, but you have tqdm 4.65.0 which is incompatible.


In [3]:
import pandas as pd

def load_excel_data(file_path: str) -> pd.ExcelFile:
    """Load Excel file from given path"""
    return pd.ExcelFile(file_path)

def parse_excel_data(excel_file: pd.ExcelFile) -> pd.DataFrame:
    """Parse Excel file into DataFrame"""
    return excel_file.parse()

def find_header_row(data: pd.DataFrame, headers: list) -> int:
    """Find the row containing any of the expected headers"""
    for idx, row in data.iterrows():
        row_str = ' '.join(str(val) for val in row.values)
        # Check if ANY header is present instead of all
        if any(header in row_str for header in headers):
            print(f"Headers found at row index: {idx}")
            return idx
    return None

def extract_metadata(data: pd.DataFrame, header_index: int) -> dict:
    """Extract and process metadata before header row"""
    metadata = data.iloc[:header_index].copy()
    metadata = metadata.dropna(how='all').reset_index(drop=True)
    
    metadata_dict = {}
    for idx, row in metadata.iterrows():
        row_clean = row.dropna()
        if not row_clean.empty:
            metadata_dict[f"row_{idx}"] = row_clean.to_dict()
    return metadata_dict

def process_rent_roll_data(data: pd.DataFrame, header_index: int) -> pd.DataFrame:
    """Process rent roll data after header row"""
    data = data.iloc[header_index:].reset_index(drop=True)
    data.columns = data.iloc[0]
    rent_roll_data = data.iloc[1:].reset_index(drop=True)
    return rent_roll_data.dropna(how='all').reset_index(drop=True)

def main():
    # Define expected headers - any one of these should be present
    headers = [
        'Resh ID', 'Lease ID', 'Unit', 'Floor Plan', 'Unit Designation', 'SQFT',
        'Unit/Lease Status', 'Name', 'Phone Number', 'Email', 'Move-In', 
        'Notice For Date', 'Move-Out', 'Lease Start', 'Lease End', 'Market + Addl.',
        'Dep On Hand', 'Balance', 'Total Charges', 'RENT', 'PEST', 'VALET TRASH',
        'TRASH', 'PETRENT', 'LOP15', 'EMPLCRED', 'OHIOCHOICE', 'WAIVED ADMIN FEE',
        'WAIVED APP FEE', 'SETUPFEE'
    ]
    
    # Pipeline execution
    file_path = input("Please enter the path to your rent roll Excel file: ")
    excel_file = load_excel_data(file_path)
    raw_data = parse_excel_data(excel_file)
    header_index = find_header_row(raw_data, headers)
    
    if header_index is not None:
        metadata = extract_metadata(raw_data, header_index)
        rent_roll_data = process_rent_roll_data(raw_data, header_index)
        
        print("Metadata:")
        print(metadata.values())
        
        return rent_roll_data
    else:
        raise ValueError("Could not find any of the expected headers in data")

# Execute pipeline
rent_roll_data = main()
rent_roll_data


Headers found at row index: 0
Metadata:
dict_values([])


,RENT ROLL DETAIL MODIFIED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,5/16/2024 3:11:55 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,As of Date: 5/15/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Parameters: Property - ALL; Unit Designation -...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Resh ID,Lease ID,Unit,Floor Plan,Unit Designation,SQFT,Unit/Lease Status,Name,Phone Number,Email,...,PEST,VALET TRASH,TRASH,PETRENT,LOP15,EMPLCRED,OHIOCHOICE,WAIVED ADMIN FEE,WAIVED APP FEE,SETUPFEE
4,101887,102845,3680,B2R,NaN,1032,Occupied,"Wray, Joe",614-815-9588,joepwrayjr@gmail.com,...,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,101854,102786,6820,B2R,NaN,1032,Occupied,"Cumberledge, Connor",480-390-0698,connorcumberledge@gmail.com,...,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
225,71258,102793,6821,B2R,NaN,1032,Occupied,"Griffith, Courtney",614-560-7706,gir9588@gmail.com,...,3.00,30.00,8.00,35.00,0.00,0.00,0.00,0.00,0.00,0.00
226,81517,102837,6822,B2R,NaN,1032,Occupied,"Ngo Nguidjoe, Persida",(240) 544-8784,pnguidjoe@gmail.com,...,3.00,30.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
227,7,102931,6823,B2R,NaN,1032,Occupied,"Morast, Lewis",614-271-7548,lewmo3118@yahoo.com,...,3.00,31.00,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
